## Day 01

In [1]:
nums = parse.(Int, readlines("01.txt"))
@assert nums[end] == 10044

In [2]:
function window_sum(idx, window_size)
    sum(nums[idx - window_size + 1:idx])
end

function count_increasing(window_size)
    result = 0
    for idx in window_size + 1:length(nums)
        if window_sum(idx, window_size) > window_sum(idx - 1, window_size)
            result += 1
        end
    end
    result
end

count_increasing (generic function with 1 method)

In [3]:
println("easy: ", count_increasing(1))
println("hard: ", count_increasing(3))

easy: 1715
hard: 1739


## Day 02

In [4]:
lines = readlines("02.txt")

function move_submarine(is_hard)
    Δx, Δy = 0, 0
    aim = 0

    for line in lines
        action, arg = split(line)
        arg = parse(Int, arg)

        if action == "forward"
            Δx += arg
            Δy += aim * arg
        else
            inc = action == "down" ? arg : -arg
            if is_hard
                aim += inc
            else
                Δy += inc
            end
        end
    end
    
    Δx * Δy
end

move_submarine (generic function with 1 method)

In [5]:
println("easy: ", move_submarine(false))
println("hard: ", move_submarine(true))

easy: 1714950
hard: 1281977850


## Day 03

`mcb` stands for Most Common Bit. 

In [6]:
lines = readlines("03.txt")
n_bits = length(lines[1])
all_nums = reduce(vcat, permutedims.(collect.(lines)))

function get_mcb(nums, bit_idx, should_invert)
    n_ones = count(==('1'), nums[:, bit_idx])
    ones_are_mcb = 2 * n_ones >= size(nums, 1)
    (ones_are_mcb ⊻ should_invert) ? '1' : '0'
end

from_chars(x) = parse(Int, join(x), base=2)

function day03_easy()
    function get_rate(should_invert)
        from_chars([
            get_mcb(all_nums, bit_idx, should_invert)
            for bit_idx in 1:n_bits
        ])
    end

    γ_rate = get_rate(false)
    ϵ_rate = get_rate(true)
    γ_rate * ϵ_rate
end

function day03_hard()
    function get_rating(should_invert)
        cur_nums = all_nums
        bit_idx = 1

        while size(cur_nums, 1) > 1
            needed_bit = get_mcb(cur_nums, bit_idx, should_invert)
            cur_nums = cur_nums[cur_nums[:, bit_idx] .== needed_bit, :]
            bit_idx += 1
        end
        
        from_chars(cur_nums)
    end
    
    o₂_rating = get_rating(false)
    co₂_rating = get_rating(true)
    o₂_rating * co₂_rating
end

println("easy: ", day03_easy())
println("hard: ", day03_hard())

easy: 2640986
hard: 6822109


## Day 04

Keep the board states in a `5 x 5 x BOARD_COUNT` array. When a number `N` is drawn, replace all occurrences of `N` with `-1`.

In [7]:
using DelimitedFiles

const N = 5

day04_file = open("04.txt")
numbers = parse.(Int, split(readline(day04_file), ','))
boards = permutedims(
    reshape(readdlm(day04_file, Int), N, :, N),
    (1, 3, 2),
)

function wins(board_idx)
    any(
        all(boards[:, idx, board_idx] .< 0) || all(boards[idx, :, board_idx] .< 0)
        for idx in 1:N
    )
end

function score(board_idx, drawn_number)
    drawn_number * sum(filter(>=(0), boards[:, :, board_idx]))
end

live_boards = Set(1:size(boards, 3))
easy_answer, hard_answer = nothing, nothing
for num in numbers
    boards[boards .== num] .= -1
    winning_boards = Set(idx for idx in live_boards if wins(idx))
    if length(winning_boards) == 1
        current_score = score(first(winning_boards), num)
        if isnothing(easy_answer)
            easy_answer = current_score
        end
        hard_answer = current_score
    end
    setdiff!(live_boards, winning_boards)
end

println("easy: ", easy_answer)
println("hard: ", hard_answer)

easy: 44088
hard: 23670


## Day 05

In [8]:
segments = open("05.txt") do f
    map(readlines(f)) do line
        split(line, " -> ") .|> x -> split(x, ",") .|> x -> parse(Int, x)
    end
end

function count_overlaps(include_diagonals)
    counts = Dict()
    
    function add_point(x, y)
        counts[(x, y)] = get!(counts, (x, y), 0) + 1
    end
    
    for ((x1, y1), (x2, y2)) in segments
        Δx, Δy = sign(x2 - x1), sign(y2 - y1)
        if include_diagonals || (Δx == 0 || Δy == 0)
            while x1 != x2 || y1 != y2
                add_point(x1, y1)
                x1 += Δx
                y1 += Δy
            end
            add_point(x1, y1)
        end
    end
    
    count(>(1), values(counts))
end


println("easy: ", count_overlaps(false))
println("hard: ", count_overlaps(true))

easy: 5585
hard: 17193


## Day 06

In [9]:
set_count(counts, idx, val) = counts[idx + 1] = val
get_count(counts, idx) = counts[idx + 1]
inc_count(counts, idx, by) = set_count(counts, idx, by + get_count(counts, idx))

function initial_counts()
    res = zeros(Int, 9)
    for f in parse.(Int, split(readline("06.txt"), ','))
        inc_count(res, f, 1)
    end
    res
end

function step(counts)
    res = [
        get_count(counts, mod(idx + 1, 9))
        for idx in 0:8
    ]
    inc_count(res, 6, get_count(counts, 0))
    res
end

function iterate_steps(n)
    res = initial_counts()
    for _ in 1:n
        res = step(res)
    end
    sum(res)
end

println("easy: ", iterate_steps(80))
println("hard: ", iterate_steps(256))

easy: 380758
hard: 1710623015163


## Day 07

In [10]:
crabs = parse.(Int, split(readline("07.txt"), ","))
lo, hi = extrema(crabs)

function get_location(cost_func)
    minimum(
        pos -> sum(
            cost_func(abs(pos - cr))
            for cr in crabs
        ),
        lo:hi
    )
end

println("easy: ", get_location(identity))
println("hard: ", get_location(x -> x * (x + 1) ÷ 2))

easy: 347449
hard: 98039527


## Day 08

0. We know the patterns for `1`, `4`, `7`, and `8`.
1. **a** is connected to the difference between the patterns for `1` and `7`.
2. `d` and `g` are connected to the common segments (excluding `a`) among 5-segment digits (`2`, `3`, `5`). **d** should be present in `4`, as opposed to **g**.
3. `b` and `d` are present in `4`, but not in `1`. We now know `d`, which means we can determine **b**.
4. The intersection of 6-segment digits (`0`, `6`, `9`) contains `a`, `b`, `f`, `g`. We can now uniquely determine **f**, since we know `a`, `b`, and `g`.
5. The intersection of `1` and `7` is `c` and `f`. We know `f`, hence we can determine **c**.
6. Finally, as we have uniquely determined every segment except `e`, we can determine what **e** is by intersecting `8` (which contains all segments) and the segments we know.

In [35]:
function parse_line(line)
    patterns, digits = split(line, " | ")
    Set.(split(patterns)), split(digits)
end

day08_input = map(parse_line, readlines("08.txt"))

function figure_out_signals(patterns)
    pattern_by_len(len) = filter(x -> length(x) == len, patterns)
    
    p1, p4, p7, p8 = map(x -> first(pattern_by_len(x)), [2, 4, 3, 7])
    p235 = pattern_by_len(5)
    p069 = pattern_by_len(6)
    
    sig = Dict()
    sig['a'] = setdiff(p7, p1)
    sig_dg = setdiff(∩(p235...), sig['a'])
    sig['d'] = ∩(sig_dg, p4)
    sig['g'] = setdiff(sig_dg, sig['d'])
    sig['b'] = setdiff(p4, p1, sig['d'])
    sig['f'] = setdiff(∩(p069...), sig['a'], sig['b'], sig['g'])
    sig['c'] = setdiff(∩(p1, p7), sig['f'])
    sig['e'] = setdiff(p8, ∪(values(sig)...))
    
    Dict(
        first(v) => k
        for (k, v) in pairs(sig)
    )
end

function get_displayed_value(patterns, digits)
    lit_segments_to_digit = Dict(
        Set("abcefg") => 0,
        Set("cf") => 1,
        Set("acdeg") => 2,
        Set("acdfg") => 3,
        Set("bcdf") => 4,
        Set("abdfg") => 5,
        Set("abdefg") => 6,
        Set("acf") => 7,
        Set("abcdefg") => 8,
        Set("abcdfg") => 9,
    )

    sig = figure_out_signals(patterns)

    join(
        map(digits) do d
            lit_segments_to_digit[Set(join(sig[ch] for ch in d))]
        end
    )
end

easy_answer, hard_answer = 0, 0
for (patterns, digits) in day08_input
    displayed = get_displayed_value(patterns, digits)
    easy_answer += sum(collect(displayed) .∈ Ref(['1', '4', '7', '8']))
    hard_answer += parse(Int, displayed)
end

println("easy: ", easy_answer)
println("hard: ", hard_answer)

easy: 310
hard: 915941


## Day 09

In [100]:
heightmap = parse.(Int, reduce(vcat, permutedims.(collect.(readlines("09.txt")))))

DELTAS = 

function get_adjacent(center)
    candidates = [
        center + CartesianIndex(delta)
        for delta in (
            (0, 1),
            (0, -1),
            (1, 0),
            (-1, 0),
        )
    ]
    filter(candidates) do adj
        checkbounds(Bool, heightmap, adj)
    end
end

function get_basin_size(center, visited)
    if center ∈ visited || heightmap[center] == 9
        return 0
    end

    push!(visited, center)

    1 + sum(
        get_basin_size(adj, visited)
        for adj in get_adjacent(center)
    )
end

easy_answer = 0
basin_sizes = []
for center in CartesianIndices(heightmap)
    risk_level = heightmap[center]
    is_low_point = all(
        heightmap[adj] > risk_level
        for adj in get_adjacent(center)
    )
    
    if is_low_point
        easy_answer += 1 + risk_level
        push!(basin_sizes, get_basin_size(center, Set()))
    end
end

sort!(basin_sizes, rev=true)

println("easy: ", easy_answer)
println("hard: ", prod(basin_sizes[1:3]))

easy: 572
hard: 847044
